In [44]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# # Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Python(82541) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
fatal: destination path 'detectron2' already exists and is not an empty directory.


Python(82544) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(82545) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [45]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

zsh:1: command not found: nvcc
torch:  2.3 ; cuda:  2.3.1
detectron2: 0.6


Python(82546) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [46]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import os, json, cv2, random
# from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [47]:
# Register Dataset
from detectron2.data.datasets import register_coco_instances
register_coco_instances("../src/coco/annotations/json_annotation_train.json", {}, "../src/coco/annotations/json_annotation_train.json", "../src/coco/images/train")
register_coco_instances("../src/coco/annotations/json_annotation_val.json", {},"../src/coco/annotations/json_annotation_val.json" , "../src/coco/images/val")

In [48]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("../src/coco/annotations/json_annotation_train.json",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.MODEL.DEVICE = "cpu"  # Force CPU usage
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

Loading config /Users/redar/PycharmProjects/summer24/axon-detection/notebooks/detectron2/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[08/11 20:32:15 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


[08/11 20:32:15 d2.engine.train_loop]: Starting training from iteration 0


Python(82557) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(82558) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


[08/11 20:34:09 d2.utils.events]:  eta: 0:24:54  iter: 19  total_loss: 3.63  loss_cls: 2.658  loss_box_reg: 1.212    time: 5.4457  last_time: 4.2307  data_time: 0.0970  last_data_time: 0.0009   lr: 1.6068e-05  
[08/11 20:36:11 d2.utils.events]:  eta: 0:21:42  iter: 39  total_loss: 1.69  loss_cls: 0.9278  loss_box_reg: 0.7337    time: 5.7876  last_time: 4.4728  data_time: 0.0022  last_data_time: 0.0011   lr: 3.2718e-05  
[08/11 20:38:08 d2.utils.events]:  eta: 0:20:02  iter: 59  total_loss: 1.074  loss_cls: 0.5188  loss_box_reg: 0.598    time: 5.8084  last_time: 4.7580  data_time: 0.0016  last_data_time: 0.0007   lr: 4.9367e-05  
[08/11 20:40:18 d2.utils.events]:  eta: 0:18:50  iter: 79  total_loss: 0.9819  loss_cls: 0.3456  loss_box_reg: 0.6288    time: 5.9745  last_time: 7.1580  data_time: 0.0016  last_data_time: 0.0010   lr: 6.6017e-05  
[08/11 20:42:09 d2.utils.events]:  eta: 0:16:47  iter: 99  total_loss: 0.8609  loss_cls: 0.2944  loss_box_reg: 0.5691    time: 5.8910  last_time: 4.

In [49]:
%load_ext tensorboard
%tensorboard --logdir output

Launching TensorBoard...

Python(83384) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [50]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

[08/11 21:11:32 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...
[Checkpointer] Loading from ./output/model_final.pth ...


In [51]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("../src/coco/annotations/json_annotation_val.json", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "../src/coco/annotations/json_annotation_val.json")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[08/11 21:11:51 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
[08/11 21:11:51 d2.data.datasets.coco]: Loaded 2 images in COCO format from ../src/coco/annotations/json_annotation_val.json
[08/11 21:11:51 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|
|    axon    | 3146         |   myelin   | 1578         |
|            |              |            |              |
|   total    | 4724         |            |              |
[08/11 21:11:51 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[08/11 21:11:51 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[08/11 21:11:51 d2.data.common]: Serializing 2 elements to byte tensors and concatenating the

Python(83425) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(83426) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


[08/11 21:11:55 d2.evaluation.evaluator]: Total inference time: 0:00:00.740754 (0.740754 s / iter per device, on 1 devices)
[08/11 21:11:55 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:00 (0.498793 s / iter per device, on 1 devices)
[08/11 21:11:55 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[08/11 21:11:55 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[08/11 21:11:55 d2.evaluation.coco_evaluation]: Evaluating predictions with official COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.19s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.018
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.032
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.018
 Average